## makedb

为每一条V germline, J germline, junction nucleotides sequence, cdr3 nucleotides sequence制作比对库

In [1]:
# load V genmline sequence
import csv

import pandas as pd
from pathlib import Path

In [7]:
# 数据目录
root = Path('/data/ab01/standard_dataset/processed_dataset')
train_data_paths = [root/f'CM00{i}'/f'CM00{i}_train_clone-pass_germ-pass.tsv' for i in [0,1,2,3]]
valid_data_paths = [root/f'CM00{i}'/f'CM00{i}_valid_clone-pass.tsv' for i in [1,2,3]]

train_data_paths

[PosixPath('/data/ab01/standard_dataset/processed_dataset/CM000/CM000_train_clone-pass_germ-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM001/CM001_train_clone-pass_germ-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM002/CM002_train_clone-pass_germ-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM003/CM003_train_clone-pass_germ-pass.tsv')]

In [8]:
valid_data_paths

[PosixPath('/data/ab01/standard_dataset/processed_dataset/CM001/CM001_valid_clone-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM002/CM002_valid_clone-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM003/CM003_valid_clone-pass.tsv')]

In [9]:
files = train_data_paths[:]
files.extend(valid_data_paths)

v_germline_alignment_set = []
j_germline_alignment_set = []
cdr3_seq_set = []
junction_seq_set = []

for file in files:
    df0 = pd.read_csv(file, sep='\t')
    df0['v_germline_len'] = df0['v_germline_alignment'].apply(lambda x: len(x))
    df0['j_germline_len'] = df0['j_germline_alignment'].apply(lambda x: len(x))
    df0['cdr3_seq_len'] = df0['cdr3'].apply(lambda x: len(x))
    df0['junction_seq_len'] = df0['junction'].apply(lambda x: len(x))
    
    v_germline_alignment_set.extend(df0['v_germline_alignment'])
    j_germline_alignment_set.extend(df0['j_germline_alignment'])
    cdr3_seq_set.extend(df0['cdr3'])
    junction_seq_set.extend(df0['junction'])
    
    df0 = df0.where(pd.notnull(df0), None)
    with open(f'./data/processed_data/{file.name}', 'w') as f:
        f_csv = csv.writer(f, delimiter='\t')
        f_csv.writerow(df0.columns)
        f_csv.writerows(df0.values)

In [14]:
print(len(junction_seq_set), len(set(junction_seq_set)))

47032 17256


In [19]:
print(len(j_germline_alignment_set))

331


In [17]:
v_germline_alignment_set = set(v_germline_alignment_set)
j_germline_alignment_set = set(j_germline_alignment_set)
cdr3_seq_set = set(cdr3_seq_set)
junction_seq_set = set(junction_seq_set)

# save V_germline
with open('data/processed_data/fasta/V_germline.fasta', 'a+') as f1:
    for i,item in enumerate(v_germline_alignment_set):
        f1.write(f'>IGHV_{i}\n')
        f1.write(item+'\n')
# save J_germline
with open('data/processed_data/fasta/J_germline.fasta', 'a+') as f2:
    for i,item in enumerate(j_germline_alignment_set):
        f2.write(f'>IGHJ_{i}\n')
        f2.write(item+'\n')
# save cdr3 nucleotide sequence
with open('data/processed_data/fasta/cdr3_sequence.fasta', 'a+') as f3:
    for i,item in enumerate(cdr3_seq_set):
        f3.write(f'>cdr3_{i}\n')
        f3.write(item+'\n')
# save junction nucleotide sequence
with open('data/processed_data/fasta/junction_sequence.fasta', 'a+') as f4:
    for i,item in enumerate(junction_seq_set):
        f4.write(f'>junction_{i}\n')
        f4.write(item+'\n')

## clonal lineage
> Reference: https://www.pnas.org/cgi/doi/10.1073/pnas.1814213116)

### step1
group same germline length V and J germline sequences using a cutoff of 90% sequence identity


### setp2

Within each group, performing single-linkage clustering on the CDR3 sequence using a cutoff of 90% sequence identity

In [ ]:
for file in files:
    df0 = pd.read_csv(file, sep='\t')
    df0['cdr3_len'] = df0['cdr3'].apply(lambda x: len(x))
    # df0['germline_v_call_without_allele'] = df0['germline_v_call'].apply(lambda x: x.split('*')[0] if x else None)
    # df0['germline_j_call_without_allele'] = df0['germline_j_call'].apply(lambda x: x.split('*')[0] if x else None)
    
    grouped = df0.groupby(by=['v_germline_alignment', 'j_germline_alignment', 'cdr3_len'])
    # grouped2 = df0.groupby(by=['germline_v_call', 'germline_j_call', 'cdr3_len'])
    df0['group_id_by_germline_seq'] = None
    
    group_id = 0
    for key, idx in grouped.groups.items():
        group_id+=1
        df0.loc[idx, 'group_id_by_germline_seq'] = group_id
    
    df0 = df0.where(pd.notnull(df0), None)
    with open(f'data/processed_data/{file.name}', 'w') as f:
        f_csv = csv.writer(f, delimiter='\t')
        f_csv.writerow(df0.columns)
        f_csv.writerows(df0.values)